# Preparing Data

In [ ]:
input (x) -> comentarios
output (y) -> Sentimientos

In [2]:
import pandas as pd


In [3]:
df_review = pd.read_csv('IMDB Dataset.csv')
df_review

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
# aqui podemos ver que el conteo de sentiment está blanceado
df_review.value_counts('sentiment')

sentiment
negative    25000
positive    25000
dtype: int64

In [5]:
# vamos a reducir la cantidad de datos para facilitar el entrenamiento del modelo
# al mismo tiempo vamos a practicas técnicas de balanceamiento
# vamos a utilizar 10000 datos en lugar de 50000
    # para ello vamos a desbalancear 9000 positivos y 1000 negativos
    
df_positivo = df_review[df_review['sentiment']=='positive'][:9000]
df_negativo = df_review[df_review['sentiment']=='negative'][:1000]

df_review_unbalanced = pd.concat([df_positivo, df_negativo])
df_review_unbalanced
 

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
...,...,...
2000,Stranded in Space (1972) MST3K version - a ver...,negative
2005,"I happened to catch this supposed ""horror"" fli...",negative
2007,waste of 1h45 this nasty little film is one to...,negative
2010,Warning: This could spoil your movie. Watch it...,negative


In [6]:
df_review_unbalanced.value_counts('sentiment')

sentiment
positive    9000
negative    1000
dtype: int64

# Unbalanced Dataset

In [7]:
!pip install imblearn

In [10]:
# reducimos los 9000 datos al nivel de 1000 para poder tratarlos correctamente
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler()
rus.fit_resample(df_review_unbalanced[['review']], df_review_unbalanced['sentiment'])

# fit_resample necesita de un objeto 2d para funcionar, y es por eso que no puede indicarse
    # input y output como simples arrays
    # por ese motivo añadimos doble corchete

(                                                 review
 0     Basically there's a family where a little boy ...
 1     This show was an amazing, fresh & innovative i...
 2     Encouraged by the positive comments about this...
 3     Phil the Alien is one of those quirky films wh...
 4     I saw this movie when I was about 12 when it c...
 ...                                                 ...
 1995  A throwback to the "old fashioned" Westerns of...
 1996  Why this is called "Mistresses" is a puzzle, b...
 1997  Gary Busey is superb in this musical biography...
 1998  It's along the line of comedy of errors, mista...
 1999  I've always believed that David and Bathsheba ...
 
 [2000 rows x 1 columns],
 0       negative
 1       negative
 2       negative
 3       negative
 4       negative
           ...   
 1995    positive
 1996    positive
 1997    positive
 1998    positive
 1999    positive
 Name: sentiment, Length: 2000, dtype: object)

In [ ]:
# podemos observar los dos outputs
    # uno asociado a cada parametro

In [11]:
# a estas dos outputs vamos a definirles un nuevo nombre de dataset como balanceado

rus = RandomUnderSampler()
df_review_bal, df_review_bal['sentiment'] = rus.fit_resample(df_review_unbalanced[['review']],
                                                             df_review_unbalanced['sentiment'])
df_review_bal.value_counts('sentiment')

# observamos como finalmente de 9000 y 1000 hemos pasado a tener 1000 x2 y 
    # de nuevo los datasets balanceados con un tamaño asequible para el entrenamiento

sentiment
negative    1000
positive    1000
dtype: int64

## Separate Data to train and test

In [12]:
# importamos libreria
from sklearn.model_selection import train_test_split
# definimos split de entrenamiento. el porcentaje puede elegirse el que se quiera,
    # así como el estado de aleatoriedad
train, test = train_test_split(df_review_bal, test_size=0.33, random_state=42)
#self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, shuffle=False, stratify=None,train_size=training_percent)

In [13]:
train

,review,sentiment
81,"I just got back from this free screening, and ...",negative
915,Claire Denis's movies seem to fall into one of...,negative
1018,CONTAINS SPOILER With the possible exception o...,positive
380,Mario Lewis of the Competitive Enterprise Inst...,negative
1029,"My first Ichikawa in many years, and the first...",positive
...,...,...
1130,i realize this review will get me bashed by th...,positive
1294,Viewed this the other night on cable on-demand...,positive
860,"Obviously, the comments above that fawn over t...",negative
1459,"Entertaining Jim Belushi vehicle, a modern coc...",positive


In [14]:
test

,review,sentiment
1860,I have to admit I am prejudiced about my vote ...,positive
353,Never saw the original movie in the series...I...,negative
1333,I have been looking for this movie for so many...,positive
905,"At the beginning of the film, you might double...",negative
1289,At the surface COOLEY HIGH is a snappy ensembl...,positive
...,...,...
118,"I have seen most, if not all of the Laurel & H...",negative
1249,"Ok, even if you can't stand Liza- this movie i...",positive
1993,Walking the tightrope between comedy and drama...,positive
522,I rented this movie with my friend for a good ...,negative


In [15]:
# antes de trabajar con los modelos vamos a definir las variables x e y de Test y Train por separado

train_x, train_y = train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']

# Transform Text data to numeric
para facilitar y optimizar el procesado de la información transformaremos los datos de texto a numerico


## Text representation (Bag of Words)
dos técnicas:

- CountVectorizer : frecuencia de aparición de una palabra en una oracion

- Tfidf : relevancia de una palabra dentro de una oración pero que no esté muy repetida en otras reviews

## Count Vectorizer

In [13]:
# para entender Count Vectorizer realizamos el siguiente ejemplo

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
text = ["Amo escribir codigo en Python. Amo el código en Python", 
        "Odio escribir codigo en Java. Odio el código en Java"]

df = pd.DataFrame({'review': ['review1', 'review2'], 'text':text})
cv = CountVectorizer()
cv_matrix = cv.fit_transform(df['text'])
df_dtm = pd.DataFrame(cv_matrix.toarray(), index=df['review'].values, columns=cv.get_feature_names())
df_dtm

# el inconveniente de este método es que devuelve las palabras mas frecuentes sin diferenciar
    # si son positivas o negativas, en nuestro proyecto, no nos va a ayudar a
    # que podamos a darle peso positivo a las palabras despues del modelado
    

C:\Users\jrver\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,amo,codigo,código,el,en,escribir,java,odio,python
review1,2,1,1,1,2,1,0,0,2
review2,0,1,1,1,2,1,2,2,0


## Tfidf (term frequency - inverse document frequency)

In [16]:
# para entender Tfidf realizamos el siguiente ejemplo

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
text = ["Amo escribir codigo en Python. Amo el código en Python", 
        "Odio escribir codigo en Java. Odio el código en Java"]

df = pd.DataFrame({'review': ['review1', 'review2'], 'text':text})
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['text'])
df_dtm = pd.DataFrame(tfidf_matrix.toarray(), index=df['review'].values, columns=tfidf.get_feature_names())
df_dtm

# en esta caso observamos los valores obtenidos en decimales que representan el procentaje
    # de peso de cada palabra
    # una palabra representativa en un review = una palabra que aparece mucho en una review pero que no aparece
        # en otros reviews  
# esta técnica destaca la frecuencia de palabra únicas en el texto y nos ofrece una mejor aproximación
    # con mayor aproximación y en nuestro proyecto será de mayor utilidad.

C:\Users\jrver\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,amo,codigo,código,el,en,escribir,java,odio,python
review1,0.576152,0.204969,0.204969,0.204969,0.409937,0.204969,0.000000,0.000000,0.576152
review2,0.000000,0.204969,0.204969,0.204969,0.409937,0.204969,0.576152,0.576152,0.000000


In [ ]:
# si comparamos estos resultados con los de Countvectorizer:
    # vemos por ejemplo que la palabra 'en' ya no está descatacada
        # esto es debido a que 'en' aparece tanto en review1 y review2 y por tanto su peso es menor
        # según esta técnica

## Transforming Text to numeric

In [16]:
# dejamos lo stopers en ingles ya que el texto real de nuestro dataset está en ingles
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

# fit busca los  mejores parametros en nuetros datos
# transform los aplica a los datos
# train_x = review
train_x_vector = tfidf.fit_transform(train_x)

# transformamos tambien los datos de test
test_x_vector = tfidf.transform(test_x)


In [17]:
# tenemos una matriz de gran tamaño de 1340 filas (reviews) y 19862 columnas (palabras)
# en muchas celdas existiran ceros porque habrá palabras que no estén
    # por eso trabajamos con una matriz sparse (una matriz dispersa), que tiene valores ceros dispersos
        # y solo 113511 son valores diferentes de 0
     # en lugar de ser una matriz comprimida
        
train_x_vector

<1340x20938 sparse matrix of type '<class 'numpy.float64'>'
	with 120173 stored elements in Compressed Sparse Row format>

# Selecting Model | Concepts

## Supervised learning
each type has different models possibilities:

- Regression (predict continuous values)
    > output = numeric

- Classification (predict discrete clase labels)(i.e. False/true, positive/negative...)
    > output categoric
    
    > Models: SVM, Decission Tree, Naive Bayes, Logistic regression

Vamos a utilizar diferentes modelos, entrenarlos y evaluar cual se ajusta mejor
ML Algorithm

1. Aprendizaje supervisado (Supervised Learning) (output numerico). Clasificacion (output discreto)
     -tanto el input como el output están definidos. 
     -se utiliza el input para inferir el output
     - input: Review
     - Output: Sentiment(discrete)
    
2. Aprendizaje No supervisado, se identifican patrones dentro del input para inferir el output

el caso de nuestro proyecto seria un caso de aprendizaje supervisado
porque sabemos los datos de input (predictor o variable independiente)
 y el output (objetivo/target o vriable dependiente)
    

# Trainning Models

### Support Vector Machines (SVM)

In [18]:

from sklearn.svm import SVC
svc = SVC(kernel='linear')
svc.fit(train_x_vector, train_y)

SVC(kernel='linear')

#### Model testing

In [19]:
print(svc.predict(tfidf.transform(['A good movie'])))
print(svc.predict(tfidf.transform(['An excellent movie'])))
print(svc.predict(tfidf.transform(['I did not like this movie at all I gave this movie away'])))

['positive']
['positive']
['negative']


### Decision Tree

In [21]:
# estamos usando datos de clasificacion
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_x_vector, train_y)

DecisionTreeClassifier()

### Naive Bayes

In [22]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_x_vector.toarray(), train_y)

GaussianNB()

### Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression 
lr = LogisticRegression()
lr.fit(train_x_vector, train_y)

LogisticRegression()

# Model evaluation
model evaluation indicators

## Model Score (Accuracy)
exactitud
este no tiene en cuenta como se distribuyen los datos


In [ ]:
# A continuación calculamos la precision de cada uno de los modelos entrenados

In [26]:
print(svc.score(test_x_vector, test_y))
print(dec_tree.score(test_x_vector, test_y))
print(gnb.score(test_x_vector.toarray(), test_y))
print(lr.score(test_x_vector, test_y))

0.8257575757575758
0.6530303030303031
0.5757575757575758
0.806060606060606


In [ ]:
# observamos que el modelo svc nos está devolviendo mayor precision que los demas

 ## F1 Score
 F1 Score=2*(Recall*Precision)/(Recall + Precision)
 Este Score tiene en cuenta como están distribuidos los datos
 Este score es mas conveniente si trabajamos con datos desbalanceados

In [ ]:
# este escore tiene encuenta dos terminos: Recall y Precision.
# precision > indica cuantos valores correctos ha escogido el modelo de una muestra con valores buscados y no buscados
# recall > indica cuantos valores correctos ha escogido el modelo del total disponible del total de datos buscados disponibles

In [28]:
# la formula f1_score pide como argumentos el valor real y (test_y)
    # junto con el valor y predicion (el modelo mas favorable fue svc, y por eso escogemos 
    # svc.predict(test_x_vector))
    
from sklearn.metrics import f1_score
f1_score(test_y, svc.predict(test_x_vector),
         labels=['positive', 'negative'],
         average=None)

# a continuacion obtenemos los scores para los comentarios positivos y los negativos
# nos da una idea la fiabilidad de nuestro modelo. siendo 1 el valor de maxima fiabilidad

array([0.82962963, 0.82170543])

# Classification report

In [29]:
from sklearn.metrics import classification_report
print(classification_report(test_y, svc.predict(test_x_vector), 
                      labels=['positive', 'negative']))

              precision    recall  f1-score   support

    positive       0.82      0.84      0.83       335
    negative       0.83      0.82      0.82       325

    accuracy                           0.83       660
   macro avg       0.83      0.83      0.83       660
weighted avg       0.83      0.83      0.83       660



# Confusion Matrix
shows the real positives and real negatives

In [30]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, svc.predict(test_x_vector), 
                      labels=['positive', 'negative'])

array([[280,  55],
       [ 60, 265]], dtype=int64)

In [ ]:
# el array obtenido la suma de todos los valores daría el total de datos del universo de datos.
# de los cuales 280 son los verdaderos positivos
# 55 son los falsos positivos
# 60 son los falsos negativos
# 265 son los verdaderos negativos

# Model optimization


## GridSearchCV

In [31]:
# declarando los parametros dentro de diccionario con dos listas
    # son parametros para buscas los mejores parametros para el modelo
    # parametro C es un parametro de penalizacion, es un termino de error que indica a la optimización
        # del modelo cuanto error es soportable
    # parametro Kernel es parte del sistema que realiza procesamiento que especifica que tipo de funcion 
        # vamos a utilizar : linear, polinomicas, rbf etc.

from sklearn.model_selection import GridSearchCV
parametros = {'C': [1,4,8,16,32], 'kernel': ['linear', 'rbf']}
svc = SVC()
svc_grid = GridSearchCV(svc, parametros, cv=5)
svc_grid.fit(train_x_vector, train_y )
                                    

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 4, 8, 16, 32], 'kernel': ['linear', 'rbf']})

In [32]:
svc_grid.best_estimator_
svc_grid.best_params_

{'C': 1, 'kernel': 'linear'}

In [33]:
 svc_grid.best_score_

0.8365671641791046

In [ ]:
# vemos que el Score es algo mayor al calculado anteriormente,
# por lo que podemos dar por concluida la optimizacion del modelo